<a href="https://colab.research.google.com/github/Saaaaangmin/LLM/blob/main/postocat_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from fastapi import FastAPI, APIRouter, HTTPException
from pydantic import BaseModel
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os
import datetime
import re
from typing import List, Dict, Optional
from dotenv import load_dotenv
import openai
import pandas as pd

# FastAPI 앱 생성
app = FastAPI()

# 환경 변수 불러오기
load_dotenv()

# 로그 전송을 위한 계정 및 서버 정보
MY_ACCOUNT = os.getenv("MY_ACCOUNT")
MY_PASSWORD = os.getenv("MY_PASSWORD")
TO_MAIL = "chlrkd0103@naver.com"
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 465

# OpenAI API 키 설정
openai.api_key = os.getenv("OPENAI_API_KEY")

# 라우터 생성
router = APIRouter()

### Log_Send 기능 ###

class LogRequest(BaseModel):
    log_date: str

def send_email(log_file_path):
    msg = MIMEMultipart()
    msg["Subject"] = "PosToCat 로그 파일입니다."
    msg["From"] = MY_ACCOUNT
    msg["To"] = TO_MAIL

    content = "안녕하세요.\nPosToCat Log파일을 전달드립니다.\n감사합니다."
    msg.attach(MIMEText(content, "plain"))

    with open(log_file_path, 'rb') as file:
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(file.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f'attachment; filename={os.path.basename(log_file_path)}')
        msg.attach(part)

    try:
        with smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT) as smtp:
            smtp.login(MY_ACCOUNT, MY_PASSWORD)
            smtp.sendmail(MY_ACCOUNT, TO_MAIL, msg.as_string())
        return True
    except smtplib.SMTPException as e:
        raise HTTPException(status_code=500, detail=f"이메일 전송 중 오류 발생: {str(e)}")

@router.post("/send-log")
async def send_log(request: LogRequest):
    log_date = request.log_date
    try:
        datetime.datetime.strptime(log_date, "%Y%m%d")
    except ValueError:
        raise HTTPException(status_code=400, detail="유효하지 않은 날짜 형식입니다. (YYYYMMDD 형식이어야 합니다.)")

    log_file_path = f"C:/NICElog/{log_date}.log"

    if not os.path.exists(log_file_path):
        raise HTTPException(status_code=404, detail=f"파일 '{log_file_path}'을 찾을 수 없습니다.")

    if send_email(log_file_path):
        return {"success": True, "message": "로그 파일이 성공적으로 전송되었습니다."}
    else:
        raise HTTPException(status_code=500, detail="로그 파일 전송 중 오류가 발생했습니다.")

### Log_Anal 기능 ###

# 로그 한 줄을 나타내는 정규 표현식
log_pattern = re.compile(
    r"^\[(?P<발생시간>\d{8} \d{2}:\d{2}:\d{2}(?: \d{3})?)\] "
    r"\[(?P<Port>\d{6})\] "
    r"\[(?P<유형>[^\]]+)\] "
    r"\[(?P<에러메시지>.*)\]$"
)

# 에러 메시지 매핑
error_descriptions = {
    "-1 port open error!": "-1 Port open error = 시리얼의 Com Port가 오픈되지 않은 상태로, 케이블 상태 및 COM Port/Baudrate 값이 올바른지 확인이 필요합니다.",
    "-3 Ack error!": "-3 Ack Error = 시리얼 데이터 ACK 수신 오류이므로, 시리얼 연동에 문제가 없는지 확인이 필요합니다.",
    "-4 lrc, cancel!": "-4 lrc, cancel = 시리얼 통신 중 LRC 체크 에러가 발생하여 응답 데이터를 수신하지 못한 상황으로, 시리얼 통신이 정상적으로 되고 있는지 확인이 필요합니다.",
    "Timeout": "Timeout Error = 응답 시간이 초과되었습니다. 연결 상태를 확인하세요.",
}

# 추적할 함수명 목록
function_names = ["ReqToCat", "ReqStop", "SetBmpFile", "SetLogDir", "GetDecSignData", "GetDllVer"]

class LogAnalysisRequest(BaseModel):
    log_date: str

class LogAnalysisResponse(BaseModel):
    발생시간: str
    Port: str
    함수명: str
    에러메시지: str

def parse_log_line(line: str) -> Optional[Dict[str, str]]:
    match = log_pattern.match(line)
    return match.groupdict() if match else None

def analyze_log(log_date: str) -> List[Dict[str, str]]:
    file_path = f"C:/NICElog/{log_date}.log"
    error_logs = []
    last_function_name = "Unknown"

    try:
        with open(file_path, 'r', encoding='cp949') as file:
            for line in file:
                parsed_line = parse_log_line(line.strip())
                if parsed_line:
                    if parsed_line['유형'].strip() in function_names:
                        last_function_name = parsed_line['유형'].strip()
                    if "ERROR" in parsed_line['유형']:
                        error_message = parsed_line['에러메시지']
                        detailed_message = error_descriptions.get(error_message, error_message)
                        error_logs.append({
                            "발생시간": parsed_line['발생시간'],
                            "Port": parsed_line['Port'],
                            "함수명": last_function_name,
                            "에러메시지": detailed_message
                        })
        if not error_logs:
            raise HTTPException(status_code=404, detail="No ERROR logs found")
        return error_logs
    except FileNotFoundError:
        raise HTTPException(status_code=404, detail=f"File not found: {file_path}")
    except PermissionError:
        raise HTTPException(status_code=403, detail=f"Permission denied for file: {file_path}")
    except UnicodeDecodeError:
        raise HTTPException(status_code=422, detail="Encoding error while reading the file")
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Unexpected error: {str(e)}")

@router.post("/analyze_log", response_model=List[LogAnalysisResponse])
async def analyze_log_endpoint(request: LogAnalysisRequest):
    return analyze_log(request.log_date)

### LLM_PosToCat 기능 ###

# Excel 데이터 불러오기
file_path = 'C:/Users/NICE/Desktop/OhSangmin/LLM/PosToCat/PosToCat_simplified_QA.xlsx'
excel_data = pd.read_excel(file_path)
history = []

class ChatRequest(BaseModel):
    question: str

class ChatResponse(BaseModel):
    answer: str

def query_openai_api(question: str) -> str:
    excel_text = excel_data.to_csv(index=False)
    full_question = f"{question}\n\n{excel_text}\n이 데이터에 기반해서 설명해줘."

    messages = [
        {"role": "system", "content": """너는 'Nice정보통신의 10년차 PosToCat 모듈 담당자야.
제공하는 파일은 CAT단말기 관련 정보 및 POS<->CAT 단말기 통신 및 VAN통신에 대해 설명되어있고, 파일 내 '질문' 열은 질문할 데이터가 있고, '응답' 열은 질문에 대한 대답이 작성되어있어.
파일의 내용을 살펴보고 이를 기반으로 답변을 줘야해. POS<->CAT단말기 간 연동 및 단말기 VAN통신에 대해 전문가처럼 답변해줘야 해.
모든 답변은 한국어로 해주고, 만약 헷갈리거나 정확한 답변이 아닐 경우 CS개발실 '02-2187-3638' 연락처로 연락해달라는 코멘트도 덧붙여줘."""}
    ]

    messages.extend(history)
    messages.append({"role": "user", "content": full_question})

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=messages,
            max_tokens=1000,
            temperature=0
        )
        answer = response.choices[0].message['content']
        history.append({"role": "user", "content": question})
        history.append({"role": "assistant", "content": answer})
        return answer
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@router.post("/chat", response_model=ChatResponse)
async def chat_endpoint(request: ChatRequest):
    answer = query_openai_api(request.question)
    return {"answer": answer}

# FastAPI 앱에 통합 라우터 추가
app.include_router(router)